In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 設定裝置為 CUDA 如果可用，否則為 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 下載並準備 MNIST 資料集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False)

# 定義神經網絡模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 平坦化
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net().to(device)

# 定義損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型
def train(model, device, trainloader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(trainloader.dataset)} ({100. * batch_idx / len(trainloader):.0f}%)]\tLoss: {loss.item():.6f}')
        torch.cuda.empty_cache()

# 測試模型
def test(model, device, testloader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # 累加批次損失
            pred = output.argmax(dim=1, keepdim=True)  # 找到概率最高的類別
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testloader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(testloader.dataset)} ({100. * correct / len(testloader.dataset):.0f}%)\n')

# 主程序
for epoch in range(1, 11):  # 訓練 10 個 epoch
    train(model, device, trainloader, optimizer, criterion, epoch)
    test(model, device, testloader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316108
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.498855
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.304533
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.351105
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.244809
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.227921
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.358895
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.243511
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.360837
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.127524

Test set: Average loss: 0.0002, Accuracy: 9399/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.141075
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.118266
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.326621
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.187633
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.247620
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.080821
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.213559
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.162606
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.156757
T

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


NameError: name 'train_dataloader' is not defined